In [ ]:
import pandas as pd
df=pd.read_excel('input_Make Groups.xlsx')
df=df.iloc[:,:3]

,Roll,Name,Email
0,1401AI01,Abhishek,Abhishek@mycollege.in
1,1401AI02,Aman,Aman@mycollege.in
2,1401AI03,Harsh,Harsh@mycollege.in
3,1401AI04,Ayush,Ayush@mycollege.in
4,1401AI05,Aditi,Aditi@mycollege.in
...,...,...,...
678,1401MT74,Jenamani,Jenamani@mycollege.in
679,1401MT75,Sadiya,Sadiya@mycollege.in
680,1401MT76,Saksham,Saksham@mycollege.in
681,1401MT77,Saharsh,Saharsh@mycollege.in


In [7]:
import os
required_cols = ["Roll", "Name", "Email"]
df['Department']=df["Roll"].str.extract(r'([A-Z]+)')
folder="full_brancwise"
for dept, group in df.groupby("Department"):
    filename = os.path.join(folder, f"{dept}.csv")
    if os.path.exists(filename):
        try:
            existing = pd.read_csv(filename)
        except pd.errors.EmptyDataError:
            existing = pd.DataFrame(columns=required_cols)
        for col in required_cols:
            if col not in existing.columns:
                existing[col] = None  
        
        existing = existing[required_cols]
    else:
        existing = pd.DataFrame(columns=["Roll", "Name", "Email"])  

    merged = pd.concat([existing, group.drop(columns="Department")], ignore_index=True)
    merged = merged.drop_duplicates(subset=["Roll"])  

    merged.to_csv(filename, index=False)

In [8]:
import pandas as pd
import os
import shutil
folder = "full_brancwise"
required_cols = ["Roll", "Name", "Email"]
def reset_output_folder(folder_path):
    """Remove old folder completely and recreate it clean."""
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)
    os.makedirs(folder_path)

departments = {}
for file in sorted(os.listdir(folder)):  # sorted ensures consistent dept order
    if file.endswith(".csv"):
        dept = file.replace(".csv", "")
        try:
            df = pd.read_csv(os.path.join(folder, file))
        except pd.errors.EmptyDataError:
            df = pd.DataFrame(columns=required_cols)

        for col in required_cols:
            if col not in df.columns:
                df[col] = None
        df = df[required_cols].dropna(subset=["Roll"])
        departments[dept] = df.reset_index(drop=True)

k = int(input("Enter number of groups: "))

groups = [[] for _ in range(k)]

row = 0
group_index = 0
while True:
    added_any = False
    for dept in departments.keys(): 
        df = departments[dept]
        if row < len(df):
            student = df.iloc[row].to_dict()
            groups[group_index].append(student)
            added_any = True
    row += 1
    group_index = (group_index + 1) % k
    if not added_any:
        break

output_folder = "group_branch_wise_mix"
reset_output_folder(output_folder)

for i, group in enumerate(groups, start=1):
    group_df = pd.DataFrame(group)
    filename = os.path.join(output_folder, f"g{i}.csv")
    group_df.to_csv(filename, index=False)


In [9]:
import pandas as pd
import os
import math
import shutil

folder = "full_brancwise"
required_cols = ["Roll", "Name", "Email"]
def reset_output_folder(folder_path):
    """Remove old folder completely and recreate it clean."""
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)
    os.makedirs(folder_path)

departments = {}
for file in sorted(os.listdir(folder)):
    if file.endswith(".csv"):
        dept = file.replace(".csv", "")
        try:
            df = pd.read_csv(os.path.join(folder, file))
        except pd.errors.EmptyDataError:
            df = pd.DataFrame(columns=required_cols)

        for col in required_cols:
            if col not in df.columns:
                df[col] = None
        df = df[required_cols].dropna(subset=["Roll"])
        departments[dept] = df.reset_index(drop=True)

k = int(input("Enter number of groups: "))

total_students = sum(len(df) for df in departments.values())
target_size = math.ceil(total_students / k)

groups = [[] for _ in range(k)]
group_sizes = [0] * k

dept_remaining = {d: df.copy() for d, df in departments.items()}

g = 0
while any(len(df) > 0 for df in dept_remaining.values()):

    dept = max(dept_remaining, key=lambda d: len(dept_remaining[d]))
    df = dept_remaining[dept]
    if len(df) == 0:
        continue

    space_left = target_size - group_sizes[g]
    take = min(space_left, len(df))

    students = df.iloc[:take].to_dict(orient="records")
    groups[g].extend(students)
    group_sizes[g] += take

    dept_remaining[dept] = df.iloc[take:].reset_index(drop=True)

    if group_sizes[g] >= target_size:
        g = (g + 1) % k

output_folder = "group_uniform_mix"
reset_output_folder(output_folder) 

for i, group in enumerate(groups, start=1):
    group_df = pd.DataFrame(group)
    filename = os.path.join(output_folder, f"g{i}.csv")
    group_df.to_csv(filename, index=False)



In [10]:
import pandas as pd
import os
import re

def make_stats_table(groups_folder):
    group_files = sorted(
        [f for f in os.listdir(groups_folder) if f.endswith(".csv")],
        key=lambda x: int(re.search(r"\d+", x).group()) if re.search(r"\d+", x) else 9999
    )
    stats = {}

    for file in group_files:
        group_name = file.replace(".csv", "").upper()
        try:
            df = pd.read_csv(os.path.join(groups_folder, file))
        except Exception:
            continue  

        if "Roll" not in df.columns:
            continue

        df["Department"] = df["Roll"].astype(str).str.extract(r"([A-Z]+)")
        counts = df["Department"].value_counts().to_dict()
        counts["Total"] = len(df)
        stats[group_name] = counts

    if stats:
        stats_df = pd.DataFrame(stats).T.fillna(0).astype(int)
        stats_df.index.name = "Group"
    else:
        stats_df = pd.DataFrame()
    return stats_df


branchwise_folder = "group_branch_wise_mix"
uniform_folder = "group_uniform_mix"
output_file = "output.xlsx"

branchwise_stats = make_stats_table(branchwise_folder)
uniform_stats = make_stats_table(uniform_folder)

# ---- Handle locked file problem ----
base_name, ext = os.path.splitext(output_file)
final_output = output_file
counter = 1

while os.path.exists(final_output):
    try:
        os.remove(final_output)   # try deleting if not locked
        break
    except PermissionError:
        # File locked → choose new name
        final_output = f"{base_name}_{counter}{ext}"
        counter += 1

# ---- Write both tables to a single sheet ----
with pd.ExcelWriter(final_output, engine="openpyxl") as writer:
    start_row = 0

    if not branchwise_stats.empty:
        pd.DataFrame([["Branch Wise Mix"]]).to_excel(
            writer, sheet_name="Statistics", startrow=start_row, header=False, index=False
        )
        start_row += 2
        branchwise_stats.to_excel(writer, sheet_name="Statistics", startrow=start_row)
        start_row += len(branchwise_stats) + 3

    if not uniform_stats.empty:
        pd.DataFrame([["Uniform Mix"]]).to_excel(
            writer, sheet_name="Statistics", startrow=start_row, header=False, index=False
        )
        start_row += 2
        uniform_stats.to_excel(writer, sheet_name="Statistics", startrow=start_row)

print(f"✅ Both stats tables written to: {final_output}")


✅ Both stats tables written to: output.xlsx
